# Solución numérica para la deformación de una viga en régimen lineal

## Formulación matemática del problema:

En este notebook se generan soluciones computacionales para la ecuación diferencial parcial (EDP) que describe la deformación elástica de un sólido cuando a este es sometido a un campo de fuerzas en el espacio, para deformaciones pequeñas <sup>(1)</sup>. La ecuación a resolver corresponde a la siguiente:

$$
\begin{align}
-\nabla\cdot\sigma &= f\hbox{ in }\Omega,\\ 
\end{align}
$$

Se definen los parámetros
$$
\begin{align}
\sigma &= \lambda\,\hbox{tr}\,(\varepsilon) I + 2\mu\varepsilon,\\ 
\varepsilon &= \frac{1}{2}\left(\nabla u + (\nabla u)^{\top}\right),
\end{align}
$$

Donde $\sigma$ es el tensor de tracción unitaria (stress) con parámetros de Lamé $\lambda$ y $\mu$ correspondientes al módulo elástico para cargas axiales y cortantes respectivamente, $\varepsilon$ es el tensor de desplazamiento unitario (Cauchy's strain tensor) y $u$ es el campo vectorial de deformaciones del cuerpo, por lo que se resuelve la EDP para $u$. Se propone la formulación variacional del problema:

$$-\int_\Omega (\nabla\cdot\sigma) \cdot v  =
\int_\Omega f\cdot v ,$$

Dado que el término $\nabla\cdot\sigma$ es de segundo orden, se puede integrar por partes de la siguiente forma:

$$
\begin{align}
-\int_\Omega (\nabla\cdot\sigma) \cdot v 
&= \int_\Omega \sigma : \nabla v - \int_{\partial\Omega}
(\sigma\cdot \hat{n})\cdot v ,
\end{align}
$$

Considerando que $\sigma \cdot \hat{n}$ corresponde al stress en la frontera del sólido, se puede caracterizar este vector como una condición de borde de Dirichlet definido como $T = \sigma \cdot \hat{n}$ en una región específica de la frontera $\partial \Omega_T \subseteq \partial \Omega$, tal que el resto de la frontera se define con una condición de Dirichlet de deformación nula. Además, es posible simplificar el producto interno $\sigma (u) : \nabla v$ como $\sigma (u) : \varepsilon (v)$, ya que la parte sólo la parte simétrica de $\nabla v$ no entrega un resultado identicamente nulo en el producto interno. La forma variacional final resulta:

$$
\int_\Omega \sigma : \varepsilon (v)  =
\int_\Omega f\cdot v 
+ \int_{\partial\Omega_T} T\cdot v,
$$

El problema se puede expresar como una ecuación expresada en una forma bilineal y otra lineal para encontrar $u\in V$:

$$
\begin{equation}
a(u,v) = L(v)\quad\forall v\in\hat{V},
\end{equation}
$$

Donde:

$$
\begin{align}
a(u,v) &= \int_\Omega\sigma(u) :\nabla v,\\ 
\sigma(u) &= \lambda(\nabla\cdot u)I + \mu(\nabla u + (\nabla u)^{\top}),\\
L(v) &= \int_\Omega f\cdot v + \int_{\partial\Omega_T}
T\cdot v,
\end{align}
$$


Formulación del problema basada en: [The equations of linear elasticity](https://fenicsproject.org/pub/tutorial/html/._ftut1008.html)

Notas al pie de página:

(1) Para encontrar soluciones para sólidos con desplazamientos mayores investigar en análisis no lineal de estructuras

## Elementos necesarios para la implementación:

La construcción del sólido a utilizar y la discretización por elementos finítos se realiza en el programa <b>Gmsh</b>, para luego importar la malla obtenida como un archivo .msh. Para poder encontrar soluciones numéricas a partir de la malla se utiliza el módulo <b>NGSolve</b>. Se define una clase para poder trabajar con varias mallas y parámetros en el problema:

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.read_gmsh import ReadGmsh

In [2]:
class Beam:
    
    def __init__(self, mesh, order, lam, mu, b, T):
        
        self.mesh = mesh
        self.order = order
        self.lam = lam
        self.mu = mu
        self.b = b
        self.T = T
        
        if mesh.dim == 3:
            self.freebnd = 'top|bottom|left|right|end'
            self.fixedbnd = 'wall'
        elif mesh.dim == 2:
            self.freebnd = 'top|bottom|right'
            self.fixedbnd = 'left'


    def define_space(self):
        
        self.V = VectorH1(self.mesh,order = self.order, dirichlet = self.fixedbnd)
        self.u = self.V.TrialFunction()
        self.v = self.V.TestFunction()
   
    def define_forms(self):
        
        global sigma
        sigma = lambda u: self.lam*Id(self.mesh.dim) * (epsilon(u)[0]+epsilon(u)[1]+epsilon(u)[2])+ 2*self.mu*epsilon(u)
        global epsilon
        epsilon = lambda u: 0.5*(grad(u) + grad(u).trans)
        
        self.a = BilinearForm(self.V)
        self.a += InnerProduct(sigma(self.u), epsilon(self.v))*dx
        self.a.Assemble()
        
        self.f = LinearForm(self.V)
        self.f += self.b*self.v*dx + self.T*self.v*ds(definedon = self.freebnd)
        self.f.Assemble()
    
    def solve_system(self):
        
        self.gfu = GridFunction(self.V)
        # se implementa la condicion de dirichlet
        self.gfu.Set(tuple([0 for _ in range(self.mesh.dim)]), definedon = mesh.Boundaries(self.fixedbnd))
        
        r = self.f.vec.CreateVector()
        r.data = self.f.vec - self.a.mat * self.gfu.vec
        self.gfu.vec.data += self.a.mat.Inverse(freedofs=self.V.FreeDofs()) * r
        
    def solve(self, draw = False):
        self.define_space()
        self.define_forms()
        self.solve_system()
        if draw:
            Draw(self.gfu, deformation = True)

Luego de definir esta clase con parámetros convenientes se realizan experimentos con los resultados a obtener.

## Resultados del modelo

En la clase Beam() se definió un modelo de elementos finitos para encontrar la deformación de una viga empotrada en 2 o 3 dimensiones considerando los parámetros modificables del problema de mayor interés. Estos son:

- mesh : malla del sólido con sus fronteras definidas para identificar las zonas empotradas y libres
- order : orden de la aproximación para el modelo
- lam ($\lambda$) : primer parámetro de Lamé, asociado con la resistencia a cargas axiales
- mu ($\mu$) : segundo parámetro de Lamé, asociado con la resistencia a cargas cortantes
- b : objeto CoefficientFunction que representa la carga por unidad de volumen dentro del sólido
- T : objeto CoefficientFunction que representa la carga por unidad de superficie en la frontera del sólido

Se realiza una primera demostración de como funciona la implementación computacional:

In [3]:
mesh = ReadGmsh('simple_brick.msh')
mesh = Mesh(mesh)
order = 2
lam = 1000
mu = 1000
b = CoefficientFunction((0,0,0))
T = CoefficientFunction((0,-0.5,0))

viga = Beam(mesh,order,lam,mu,b,T)
viga.solve(draw = True)

print(min(viga.gfu.vec))

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

-1.477552735808667


Como se puede apreciar, la viga reacciona a través de deformaciones (en este caso elásticas y lineales) a las cargas aplicadas, es posible modificar el empotramiento de la viga al cambiar los atributos viga.freebnd y viga.fixedbnd para superficies libres y empotradas respectivamente. El formato de entrada de este parámetro es un string regex con los nombres de los grupos físicos separados por el carácter "|".

## Análisis: parámetros de Lamé

Los parámetros de Lamé en la mecánica del continuo corresponden a dos magnitudes que describen las propiedades mecánicas para la deformación de un material ante la presencia de cargas. Se encuentran presentes en la ley de Hooke en tres dimensiones:

$$
\sigma = \lambda\,\hbox{tr}\,(\varepsilon) I + 2\mu\varepsilon,
$$


Se relacionan con el módulo elástico para esfuerzos axiales ($E$) y cortantes ($G$) de la siguiente manera:

$$
\begin{align}
E&=\mu \frac{3\lambda + 2\mu}{\lambda + \mu},\\
G&=\mu,
\end{align}
$$

En el siguiente ejemplo se modeliza como una barra de acero de 15x1x1 metros se ve afectada por su peso propio:

In [8]:
mesh = ReadGmsh('steel_bar.msh')
mesh = Mesh(mesh)
order = 3

# parámetros de Lamé para el acero (aproximados)
lam = 76000
mu = 85000

b = CoefficientFunction((0,-7850*9.8*10**-6,0)) # peso propio
T = CoefficientFunction((0,0,0)) # carga externa

viga = Beam(mesh,order,lam,mu,b,T)
viga.freebnd = 'top'
viga.solve(draw = True)



print(f'La deflexión máxima es de: {min(viga.gfu.vec)}')

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

La deflexión máxima es de: -1.1925206993108028


Los resultados de deflexión se pueden comparar con los planteados por la teoría de Euler-Bernoulli para vigas de una dimensión con el objetivo de comparar ordenes de magnitud. Según las ecuaciones de deformación en una dimensión:

$$
\varepsilon_{max} = \frac{qL^4}{8EI} = -0.0278
$$

Esta diferencia puede deberse a las diferencias en la formulación matemática del problema, pero entrega indicios de que ambos métodos entregan resultados similares.

Por otro lado, esta viga se encuentra tiene una longitud (span) mayor al permitido por las normas estructurales convencionales, por lo que se pueden apreciar pequeñas deformaciones hacia el extremo final del sólido. Al variar los parámetros $\lambda$ y $\mu$ manteniendo las cargas y teniendo en cuenta que esta ecuación rige para deformaciones pequeñas, se obtienen los siguientes resultados:

In [13]:
import numpy as np
import pandas as pd

In [14]:
llam = [60000, 75000, 90000]
lmu =  [70000, 85000, 100000]

order = 2
b = CoefficientFunction((0,-3,0)) # peso propio
T = CoefficientFunction((0,0,0)) # carga externa

i_lam = 0
max_strain = np.zeros((3,3))
for lam in llam:
    i_mu = 0
    for mu in lmu:
        viga = Beam(mesh,order,lam,mu,b,T)
        viga.freebnd = 'top'
        viga.solve()
        
        max_strain[i_lam,i_mu] = min(viga.gfu.vec)
        
        i_mu += 1
    i_lam += 1

df = pd.DataFrame(max_strain,
                 index = pd.Index([f'$\lambda$ = {llam[0]}',f'$\lambda$ = {llam[1]}',f'$\lambda$ = {llam[2]}']),
                 columns = pd.Index([f'$\mu$ = {lmu[0]}',f'$\mu$ = {lmu[1]}',f'$\mu$  = {lmu[2]}']))

Draw(viga.gfu, deformation = True)

print('Valores para deformación unitaria respecto a parámetros de Lamé:')
df

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Valores para deformación unitaria respecto a parámetros de Lamé:


,$\mu$ = 70000,$\mu$ = 85000,$\mu$ = 100000
$\lambda$ = 60000,-1.143889,-0.994708,-0.879906
$\lambda$ = 75000,-1.063895,-0.933776,-0.831939
$\lambda$ = 90000,-0.994155,-0.879773,-0.788872


Se puede apreciar que para una distribución donde el peso propio es dominante, el módulo de elasticidad transversal $\mu$ posee una mayor importancia relativa respecto al parámetro $\lambda$, ya que el esfuerzo de la viga es mayoritariamente de momento flector.

Se realiza un experimento con otra distribución de cargas para poder comparar la importancia relativa de los parámetros en cada prueba. La distribución de cargas será de tensión, aplicada en la cara del extremo final de la viga:

In [15]:
llam = [60000, 75000, 90000]
lmu =  [70000, 85000, 100000]

order = 2
b = CoefficientFunction((0,0,0)) # peso propio
T = CoefficientFunction((10000,0,0)) # carga externa

i_lam = 0
max_strain = np.zeros((3,3))
for lam in llam:
    i_mu = 0
    for mu in lmu:
        viga = Beam(mesh,order,lam,mu,b,T)
        viga.freebnd = 'end'
        viga.solve()
        
        max_strain[i_lam,i_mu] = max(viga.gfu.vec)
        
        i_mu += 1
    i_lam += 1

df = pd.DataFrame(max_strain,
                 index = pd.Index([f'$\lambda$ = {llam[0]}',f'$\lambda$ = {llam[1]}',f'$\lambda$ = {llam[2]}']),
                 columns = pd.Index([f'$\mu$ = {lmu[0]}',f'$\mu$ = {lmu[1]}',f'$\mu$  = {lmu[2]}']))

Draw(viga.gfu,deformation = True)

print('Valores para deformación unitaria respecto a parámetros de Lamé:')
df

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Valores para deformación unitaria respecto a parámetros de Lamé:


,$\mu$ = 70000,$\mu$ = 85000,$\mu$ = 100000
$\lambda$ = 60000,0.751564,0.653131,0.577552
$\lambda$ = 75000,0.699841,0.613590,0.546349
$\lambda$ = 90000,0.654964,0.578677,0.518419


Se puede ver que en el experimento de tracción pura ambos parámetros tienen efectos significativos en la deformación máxima del material, esto es porque el módulo elástico para cargas axiales ($E$) depende de ambos parámetros.

También es posible generar tensiones de corte con el parámetro de esfuerzo superficial definido como condición de borde en la ecuación. Se analiza el caso en que la carga neta genera este tipo de tensiones en la barra:

In [16]:
llam = [60000, 75000, 90000]
lmu =  [70000, 85000, 100000]
order = 2


from math import pi

b = CoefficientFunction((0,0,0))
T = CoefficientFunction((0,5000*cos(pi*(0.5-z)),5000*cos(pi*(0.5-y))))

i_lam = 0
max_strain = np.zeros((3,3))
for lam in llam:
    i_mu = 0
    for mu in lmu:
        viga = Beam(mesh,order,lam,mu,b,T)
        viga.freebnd = 'top|bottom|left|right'
        viga.solve()
        max_strain[i_lam,i_mu] = max(viga.gfu.vec)
        
        i_mu += 1
    i_lam += 1

df = pd.DataFrame(max_strain,
                 index = pd.Index([f'$\lambda$ = {llam[0]}',f'$\lambda$ = {llam[1]}',f'$\lambda$ = {llam[2]}']),
                 columns = pd.Index([f'$\mu$ = {lmu[0]}',f'$\mu$ = {lmu[1]}',f'$\mu$  = {lmu[2]}']))

Draw(viga.gfu, deformation = True)

print('Valores para deformación unitaria respecto a parámetros de Lamé:')
df

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Valores para deformación unitaria respecto a parámetros de Lamé:


,$\mu$ = 70000,$\mu$ = 85000,$\mu$ = 100000
$\lambda$ = 60000,0.111201,0.082428,0.064286
$\lambda$ = 75000,0.125782,0.093044,0.072387
$\lambda$ = 90000,0.139248,0.102906,0.079953


Se puede ver que la magnitud de la deformación depende mayoritariamente del parámetro $\mu$, ya que este determina el módulo elástico para tensiones de corte, aunque el parámetro $\lambda$ también influye en la deformación del sólido en menor medida.

## Shape Optimization con método del gradiente conjugado

Se puede definir la función objetivo del problema como la energía de deformación del sólido, esta se puede calcular de la siguiente manera:

$$
E_{def} = \frac{1}{2} \int_{\Omega} \sigma : \varepsilon
$$

Notar que esta es equivalente a la forma bilineal del problema, lo que tiene sentido ya que la formulación de la ecuación se basa en el principio de trabajos virtuales. Se puede proponer el siguiente problema de optimización en el que se minimiza la energía con la restricción de que se debe cumplir la ecuación diferencial:

\begin{equation}
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega \sigma : \varepsilon \; dx,
\end{equation}
 Sujeto a que $(\Omega,u)$ satisfacen
 \begin{equation}
        \int_\Omega \sigma : \varepsilon (v)  = \int_\Omega f\cdot v + \int_{\partial\Omega_T} T\cdot v,
\end{equation}

Esto se implementa computacionalmente utilizando las herramientas de shape optimization de NGSolve.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from ngsolve.solvers import *

Se genera una malla bidimensional a partir del módulo de geometría spline: 

In [2]:
geo = SplineGeometry()

# definición de puntos
p1 = geo.AppendPoint (0,0)
p2 = geo.AppendPoint (10,0)
p3 = geo.AppendPoint (10,1)
p4 = geo.AppendPoint (0,1)

# unión de los puntos en lineas
geo.Append (["line", p1, p2], bc = 'bottom')
geo.Append (["line", p2, p3], bc = 'right')
geo.Append (["line", p3, p4], bc = 'top')
geo.Append (["line", p4, p1], bc = 'left')

# se generan los elementos y nodos
mesh = geo.GenerateMesh (maxh=0.25)

# se define el objeto de ngsolve
mesh = Mesh(mesh)

Se define el espacio de funciones a utilizar, en este caso vectoriales derivables. Para generar la solución de la ecuación diferencial, así como el gradiente de la forma (shape differentiation) se generan dos funciones en el espacio, <i>gfu</i> corresponde a la solución para la ecuación diferencial y <i>gfp</i> corresponde a un término de la derivada direccional, que será expresado en términos de la función de costo:

In [3]:
fes = VectorH1(mesh, order=2, dirichlet = 'left|right')
gfu = GridFunction(fes)
gfp = GridFunction(fes)

Se definen las funciones y operadores que se utilizan en la formulación del modelo:

In [4]:
lam = 1000
mu = 1000


def sigma(u):
    return lam*Id(mesh.dim) * (epsilon(u)[0]+epsilon(u)[1]+epsilon(u)[2])+ 2*mu*epsilon(u)

def epsilon(u):
    return 0.5*(grad(u) + grad(u).trans)

    
def EquationFA(u,v):
    return InnerProduct(sigma(u), epsilon(v))*dx

def CostAutoFA(u): 
    return InnerProduct(sigma(u),epsilon(u))*dx

def CostAuto2(u): 
    return CostAutoFA(u)

def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - aAuto.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = aAuto.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update    

LagrangianFA = CostAutoFA(gfu) + EquationFA(gfu,gfp)

In [5]:
u, v = fes.TnT()

b = CoefficientFunction((0,-1))

aAuto = BilinearForm(fes)
aAuto += EquationFA(u,v) - (b*v)*dx # forma bilineal para el método de newton
aAuto.Assemble()

p, w = fes.TnT()

fadjoint = LinearForm(fes)
fadjoint += -1*(CostAutoFA(gfu)).Diff(gfu,w)

VEC = VectorH1(mesh, order=2)
PHI, X = VEC.TnT()


dJOmegaAuto = LinearForm(VEC)
dJOmegaAuto += LagrangianFA.DiffShape(X)

b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

Newton(aAuto, gfu, fes.FreeDofs(), printing = False)

Draw(gfu, deformation = True)

gfset = GridFunction(VEC)
gfset.Set((0,0))

gfu.Set((0,0))
scene_u = Draw (gfu, mesh, "state", deformation = True)

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output


#reset to and solve for initial configuration
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene_u.Redraw()
gfu.vec[:]=0
Newton(aAuto, gfu, fes.FreeDofs())
LineSearch = False


##################
ShowObj = True
VTKexport = True
##################


if VTKexport:
    # VTKOutput object
    vtk = VTKOutput(ma = mesh,
                    coefs = [gfu],
                    names = ['deformacion'],
                    filename = 'optimization/ambos_lados',
                    subdivision = 1)
    # Exporting the results:
    vtk.Do()

iter_max = 1000
Jold = Integrate(CostAuto2(gfu), mesh)
converged = False


if ShowObj:
    iteracion = []
    objetivo = []

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)


for k in range(iter_max):
    #print('cost at iteration', k, ': ', Jold)
    
    if ShowObj:
        iteracion.append(k)
        objetivo.append(Jold)

        ax.set_xlim(0,k)
        ax.cla()
        ax.plot(iteracion, objetivo)
        display(fig)
        clear_output(wait = True)
    
    mesh.SetDeformation(gfset)
    #
    #mesh.ngmesh.OptimizeMesh2d()
    #
    scene_u.Redraw()
    
    gfu.vec[:]=0
    Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmegaAuto.Assemble()
    SolveDeformationEquationAuto()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec

    gfset.components[0].Set(0, definedon = 'left') # "component-wise" neumann boundary condition
    gfset.components[0].Set(0, definedon = 'right')
    
    Jnew = Integrate(CostAuto2(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            scale = scale / 2
            
            if scale <= 1e-7:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            gfu.vec[:]=0
            Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
            Jnew = Integrate(CostAuto(gfu), mesh)
    
    if converged==True:
        break
    Jold = Jnew
    
    if VTKexport:
        vtk.Do()
    
    Redraw(blocking=True)

KeyboardInterrupt: 